In [1]:
// Create a map to hold the possible directions and the changes at intersections (x,y)
val directionsMap = Map(
    ("NORTH"->(-1,0)),
    ("EAST"->(0,1)),
    ("SOUTH"->(1,0)),
    ("WEST"->(0,-1))
)
val directions = Seq("NORTH", "EAST", "SOUTH", "WEST")
val intersectionChanges = Seq(-1, 0, 1)
// Also map cart faces to directions and (track to cart faces)?
val cartFaces = Map(
    ('^'->"NORTH"),
    ('>'->"EAST"),
    ('v'->"SOUTH"),
    ('<'->"WEST"))
val cartRenderChars = Map(
    ("NORTH"->'^'),
    ("EAST"->'>'),
    ("SOUTH"->'v'),
    ("WEST"->'<')
)
val cartReplacementChars = Map(
    ("NORTH"->'|'),
    ("EAST"->'-'),
    ("SOUTH"->'|'),
    ("WEST"->'-')
)

directionsMap: Map[String, (Int, Int)] = Map(
  "NORTH" -> (-1, 0),
  "EAST" -> (0, 1),
  "SOUTH" -> (1, 0),
  "WEST" -> (0, -1)
)
directions: Seq[String] = List("NORTH", "EAST", "SOUTH", "WEST")
intersectionChanges: Seq[Int] = List(-1, 0, 1)
cartFaces: Map[Char, String] = Map(
  '^' -> "NORTH",
  '>' -> "EAST",
  'v' -> "SOUTH",
  '<' -> "WEST"
)
cartRenderChars: Map[String, Char] = Map(
  "NORTH" -> '^',
  "EAST" -> '>',
  "SOUTH" -> 'v',
  "WEST" -> '<'
)
cartReplacementChars: Map[String, Char] = Map(
  "NORTH" -> '|',
  "EAST" -> '-',
  "SOUTH" -> '|',
  "WEST" -> '-'
)

In [2]:
// Create a class for the carts
class Cart extends Ordered[Cart]{
    private var facing = "NORTH"
    private var intersectionChangeIndex = 0
    // These are backwards in tuples because of 2D Array indexing/reading
    private var posX = 0
    private var posY = 0
    
    def this(facing: String, posX: Int, posY: Int) = {
        this();
        this.facing = facing;
        this.posX = posX;
        this.posY = posY;
    }
    
    // Update the current position based on the facing direction
    def move() : Unit = {
        val (y,x) = directionsMap(facing)
        this.posX += x
        this.posY += y
    }
    
    // Turn the cart 90 degrees to the left
    def turnLeft() : Unit = {
        facing = directions((directions.indexOf(facing) - 1 + directions.length) % directions.length)
    }
    
    // Turn the cart 90 degrees to the left
    def turnRight() : Unit = {
        facing = directions((directions.indexOf(facing) + 1) % directions.length)
    }
    
    // Turn the cart depending on how this cart will behave at the next intersection
    def turnIntersection : Unit = {
        facing = directions((directions.indexOf(facing) 
                  + intersectionChanges(intersectionChangeIndex) 
                  + directions.length) % directions.length)
        intersectionChangeIndex = (intersectionChangeIndex + 1) % intersectionChanges.length
    }
    
    // Draw the current character using the facing var
    def render() : Char = {
        cartRenderChars(facing)
    }
    
    // Compare positions with another cart - true if crashed
    def checkCrashed(that: Cart) : Boolean = {
        posX == that.getX && posY == that.getY
    }
    
    override def toString(): String = 
    {
        val cartChar = render()
        s"$cartChar - $facing at ($posX, $posY)"
    }
    
    // Comparison for ordering the carts
    override def compare(that: Cart): Int = {
        if (posY == that.getY) 
             posX - that.getX else
             posY - that.getY
    }
    
    def getX() : Int = {posX}
    def getY() : Int = {posY}
    def getFacing() : String = {facing}
}

defined class Cart

In [30]:
// Read in the grid
import scala.collection.mutable.ListBuffer
import scala.io.Source

val filename = "input"
val grid = Source.fromFile(filename).getLines.toArray.map(_ toCharArray)
var seconds = 0
var crashes = ListBuffer[(Int, Int)]()

import scala.collection.mutable.ListBuffer
import scala.io.Source
filename: String = "input"
grid: Array[Array[Char]] = Array(
  Array(
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
...
seconds: Int = 0
crashes: ListBuffer[(Int, Int)] = ListBuffer()

In [31]:
// Store the carts in a structure and initialise their positions/directions
// Remap the grid to remove the carts and replace them with valid grid sections
val carts = ListBuffer[Cart]()
for {
    y <- 0 until grid.length
    x <- 0 until grid(0).length } {
    val gridChar = grid(y)(x)
    if (cartFaces.contains(gridChar)) {
        carts += new Cart(cartFaces(gridChar), x, y)
        grid(y)(x) = cartReplacementChars(cartFaces(gridChar))
    }
}


carts: ListBuffer[Cart] = ListBuffer(
  < - WEST at (105, 3),
  > - EAST at (105, 22),
  < - WEST at (90, 40),
  ^ - NORTH at (27, 42),
  < - WEST at (130, 46),
  > - EAST at (82, 67),
  > - EAST at (98, 67),
  v - SOUTH at (66, 71),
  v - SOUTH at (136, 92),
  v - SOUTH at (132, 94),
  v - SOUTH at (26, 99),
  ^ - NORTH at (94, 100),
  > - EAST at (67, 104),
  ^ - NORTH at (81, 105),
...

In [32]:
def printGrid(grid : Array[Array[Char]], carts : ListBuffer[Cart], crashes : ListBuffer[(Int, Int)]) : Unit = {
    val newGrid = grid.map(_.clone)
    carts.foreach(cart => {
        newGrid(cart.getY)(cart.getX) = cart.render()})
    crashes.foreach{case (x, y) => {
        newGrid(y)(x) = 'X'}}
    for (row <- newGrid) {
        for (rowChar <- row) {
            print(rowChar)
        }
        println()
    }
}

defined function printGrid

In [33]:
println("Seconds: ", seconds)
//printGrid(grid, carts, crashes)
while (crashes.length == 0) {
    seconds += 1
    println("Seconds: ", seconds)
    carts.sorted.foreach(cart => {
        // Move and turn if necessary
        cart.move()
        val cartPos = grid(cart.getY)(cart.getX)
        var cartFacing = cart.getFacing
        cartPos match {
            case '/' => {
                cartFacing match {
                    case "NORTH" => {cart.turnRight}
                    case "EAST" => {cart.turnLeft}
                    case "SOUTH" => {cart.turnRight}
                    case "WEST" => {cart.turnLeft}
                }
            }
            case '\\' => {
                cartFacing match {
                    case "NORTH" => {cart.turnLeft}
                    case "EAST" => {cart.turnRight}
                    case "SOUTH" => {cart.turnLeft}
                    case "WEST" => {cart.turnRight}
                }
            }
            case '+' => {
                cart.turnIntersection
            }
            case _ => {}
        }
        // Check for crashes here, as cart movements are ordered
        for (otherCart <- carts) {
        if (cart != otherCart && cart.checkCrashed(otherCart)) {
            crashes += ((cart.getX, cart.getY))
            carts -= cart
            carts -= otherCart
            // Not updating the grid here, but probably should make it impassible?
            }
        }
    })
    
    //if (seconds > 170) {printGrid(grid, carts, crashes)}
    //printGrid(grid, carts, crashes)
    //carts.foreach(println)
}
crashes

(Seconds: ,0)
(Seconds: ,1)
(Seconds: ,2)
(Seconds: ,3)
(Seconds: ,4)
(Seconds: ,5)
(Seconds: ,6)
(Seconds: ,7)
(Seconds: ,8)
(Seconds: ,9)
(Seconds: ,10)
(Seconds: ,11)
(Seconds: ,12)
(Seconds: ,13)
(Seconds: ,14)
(Seconds: ,15)
(Seconds: ,16)
(Seconds: ,17)
(Seconds: ,18)
(Seconds: ,19)
(Seconds: ,20)
(Seconds: ,21)
(Seconds: ,22)
(Seconds: ,23)
(Seconds: ,24)
(Seconds: ,25)
(Seconds: ,26)
(Seconds: ,27)
(Seconds: ,28)
(Seconds: ,29)
(Seconds: ,30)
(Seconds: ,31)
(Seconds: ,32)
(Seconds: ,33)
(Seconds: ,34)
(Seconds: ,35)
(Seconds: ,36)
(Seconds: ,37)
(Seconds: ,38)
(Seconds: ,39)
(Seconds: ,40)
(Seconds: ,41)
(Seconds: ,42)
(Seconds: ,43)
(Seconds: ,44)
(Seconds: ,45)
(Seconds: ,46)
(Seconds: ,47)
(Seconds: ,48)
(Seconds: ,49)
(Seconds: ,50)
(Seconds: ,51)
(Seconds: ,52)
(Seconds: ,53)
(Seconds: ,54)
(Seconds: ,55)
(Seconds: ,56)
(Seconds: ,57)
(Seconds: ,58)
(Seconds: ,59)
(Seconds: ,60)
(Seconds: ,61)
(Seconds: ,62)
(Seconds: ,63)
(Seconds: ,64)
(Seconds: ,65)
(Seconds: ,66)
(Seco

res25_2: ListBuffer[(Int, Int)] = ListBuffer((64, 57))

In [34]:
crashes
seconds
//printGrid(grid, carts, crashes)
carts.length

res26_0: ListBuffer[(Int, Int)] = ListBuffer((64, 57))
res26_1: Int = 124
res26_2: Int = 15

## Part 2

In [3]:
// Read in the grid
import scala.collection.mutable.ListBuffer
import scala.io.Source

val filename = "input"
val grid = Source.fromFile(filename).getLines.toArray.map(_ toCharArray)
var seconds = 0
var crashes = ListBuffer[(Int, Int)]()

import scala.collection.mutable.ListBuffer
import scala.io.Source
filename: String = "input"
grid: Array[Array[Char]] = Array(
  Array(
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
    ' ',
...
seconds: Int = 0
crashes: collection.mutable.ListBuffer[(Int, Int)] = ListBuffer()

In [4]:
// Store the carts in a structure and initialise their positions/directions
// Remap the grid to remove the carts and replace them with valid grid sections
val carts = ListBuffer[Cart]()
for {
    y <- 0 until grid.length
    x <- 0 until grid(0).length } {
    val gridChar = grid(y)(x)
    if (cartFaces.contains(gridChar)) {
        carts += new Cart(cartFaces(gridChar), x, y)
        grid(y)(x) = cartReplacementChars(cartFaces(gridChar))
    }
}


carts: ListBuffer[Cart] = ListBuffer(
  < - WEST at (105, 3),
  > - EAST at (105, 22),
  < - WEST at (90, 40),
  ^ - NORTH at (27, 42),
  < - WEST at (130, 46),
  > - EAST at (82, 67),
  > - EAST at (98, 67),
  v - SOUTH at (66, 71),
  v - SOUTH at (136, 92),
  v - SOUTH at (132, 94),
  v - SOUTH at (26, 99),
  ^ - NORTH at (94, 100),
  > - EAST at (67, 104),
  ^ - NORTH at (81, 105),
...

In [5]:
def printGrid(grid : Array[Array[Char]], carts : ListBuffer[Cart], crashes : ListBuffer[(Int, Int)]) : Unit = {
    val newGrid = grid.map(_.clone)
    carts.foreach(cart => {
        newGrid(cart.getY)(cart.getX) = cart.render()})
    //crashes.foreach{case (x, y) => {
    //    newGrid(y)(x) = 'X'}}
    for (row <- newGrid) {
        for (rowChar <- row) {
            print(rowChar)
        }
        println()
    }
}

defined function printGrid

In [6]:
println("Seconds: ", seconds)
//printGrid(grid, carts, crashes)
while (carts.length > 1) {
    seconds += 1
    println("Seconds: ", seconds)
    println("NumCarts: ", carts.length)
    carts.sorted.foreach(cart => {
        // Move and turn if necessary
        cart.move()
        val cartPos = grid(cart.getY)(cart.getX)
        var cartFacing = cart.getFacing
        cartPos match {
            case '/' => {
                cartFacing match {
                    case "NORTH" => {cart.turnRight}
                    case "EAST" => {cart.turnLeft}
                    case "SOUTH" => {cart.turnRight}
                    case "WEST" => {cart.turnLeft}
                }
            }
            case '\\' => {
                cartFacing match {
                    case "NORTH" => {cart.turnLeft}
                    case "EAST" => {cart.turnRight}
                    case "SOUTH" => {cart.turnLeft}
                    case "WEST" => {cart.turnRight}
                }
            }
            case '+' => {
                cart.turnIntersection
            }
            case _ => {}
        }
        // Check for crashes here, as cart movements are ordered
        for (otherCart <- carts) {
        if (cart != otherCart && cart.checkCrashed(otherCart)) {
            crashes += ((cart.getX, cart.getY))
            carts -= cart
            carts -= otherCart
            }
        }
    })
    
    //if (seconds > 170) {printGrid(grid, carts, crashes)}
    //printGrid(grid, carts, crashes)
    //carts.foreach(println)
}
crashes

(Seconds: ,0)
(Seconds: ,1)
(NumCarts: ,17)
(Seconds: ,2)
(NumCarts: ,17)
(Seconds: ,3)
(NumCarts: ,17)
(Seconds: ,4)
(NumCarts: ,17)
(Seconds: ,5)
(NumCarts: ,17)
(Seconds: ,6)
(NumCarts: ,17)
(Seconds: ,7)
(NumCarts: ,17)
(Seconds: ,8)
(NumCarts: ,17)
(Seconds: ,9)
(NumCarts: ,17)
(Seconds: ,10)
(NumCarts: ,17)
(Seconds: ,11)
(NumCarts: ,17)
(Seconds: ,12)
(NumCarts: ,17)
(Seconds: ,13)
(NumCarts: ,17)
(Seconds: ,14)
(NumCarts: ,17)
(Seconds: ,15)
(NumCarts: ,17)
(Seconds: ,16)
(NumCarts: ,17)
(Seconds: ,17)
(NumCarts: ,17)
(Seconds: ,18)
(NumCarts: ,17)
(Seconds: ,19)
(NumCarts: ,17)
(Seconds: ,20)
(NumCarts: ,17)
(Seconds: ,21)
(NumCarts: ,17)
(Seconds: ,22)
(NumCarts: ,17)
(Seconds: ,23)
(NumCarts: ,17)
(Seconds: ,24)
(NumCarts: ,17)
(Seconds: ,25)
(NumCarts: ,17)
(Seconds: ,26)
(NumCarts: ,17)
(Seconds: ,27)
(NumCarts: ,17)
(Seconds: ,28)
(NumCarts: ,17)
(Seconds: ,29)
(NumCarts: ,17)
(Seconds: ,30)
(NumCarts: ,17)
(Seconds: ,31)
(NumCarts: ,17)
(Seconds: ,32)
(NumCarts: ,17)
(Se

(Seconds: ,262)
(NumCarts: ,15)
(Seconds: ,263)
(NumCarts: ,15)
(Seconds: ,264)
(NumCarts: ,15)
(Seconds: ,265)
(NumCarts: ,15)
(Seconds: ,266)
(NumCarts: ,15)
(Seconds: ,267)
(NumCarts: ,15)
(Seconds: ,268)
(NumCarts: ,15)
(Seconds: ,269)
(NumCarts: ,15)
(Seconds: ,270)
(NumCarts: ,15)
(Seconds: ,271)
(NumCarts: ,15)
(Seconds: ,272)
(NumCarts: ,15)
(Seconds: ,273)
(NumCarts: ,15)
(Seconds: ,274)
(NumCarts: ,15)
(Seconds: ,275)
(NumCarts: ,15)
(Seconds: ,276)
(NumCarts: ,15)
(Seconds: ,277)
(NumCarts: ,15)
(Seconds: ,278)
(NumCarts: ,15)
(Seconds: ,279)
(NumCarts: ,15)
(Seconds: ,280)
(NumCarts: ,15)
(Seconds: ,281)
(NumCarts: ,15)
(Seconds: ,282)
(NumCarts: ,15)
(Seconds: ,283)
(NumCarts: ,15)
(Seconds: ,284)
(NumCarts: ,15)
(Seconds: ,285)
(NumCarts: ,15)
(Seconds: ,286)
(NumCarts: ,15)
(Seconds: ,287)
(NumCarts: ,15)
(Seconds: ,288)
(NumCarts: ,15)
(Seconds: ,289)
(NumCarts: ,15)
(Seconds: ,290)
(NumCarts: ,15)
(Seconds: ,291)
(NumCarts: ,15)
(Seconds: ,292)
(NumCarts: ,15)
(Seconds

(Seconds: ,521)
(NumCarts: ,11)
(Seconds: ,522)
(NumCarts: ,11)
(Seconds: ,523)
(NumCarts: ,11)
(Seconds: ,524)
(NumCarts: ,9)
(Seconds: ,525)
(NumCarts: ,9)
(Seconds: ,526)
(NumCarts: ,9)
(Seconds: ,527)
(NumCarts: ,9)
(Seconds: ,528)
(NumCarts: ,9)
(Seconds: ,529)
(NumCarts: ,9)
(Seconds: ,530)
(NumCarts: ,9)
(Seconds: ,531)
(NumCarts: ,9)
(Seconds: ,532)
(NumCarts: ,9)
(Seconds: ,533)
(NumCarts: ,9)
(Seconds: ,534)
(NumCarts: ,9)
(Seconds: ,535)
(NumCarts: ,9)
(Seconds: ,536)
(NumCarts: ,9)
(Seconds: ,537)
(NumCarts: ,9)
(Seconds: ,538)
(NumCarts: ,9)
(Seconds: ,539)
(NumCarts: ,9)
(Seconds: ,540)
(NumCarts: ,9)
(Seconds: ,541)
(NumCarts: ,9)
(Seconds: ,542)
(NumCarts: ,9)
(Seconds: ,543)
(NumCarts: ,9)
(Seconds: ,544)
(NumCarts: ,9)
(Seconds: ,545)
(NumCarts: ,9)
(Seconds: ,546)
(NumCarts: ,9)
(Seconds: ,547)
(NumCarts: ,9)
(Seconds: ,548)
(NumCarts: ,9)
(Seconds: ,549)
(NumCarts: ,9)
(Seconds: ,550)
(NumCarts: ,9)
(Seconds: ,551)
(NumCarts: ,9)
(Seconds: ,552)
(NumCarts: ,9)
(Seco

(NumCarts: ,9)
(Seconds: ,789)
(NumCarts: ,9)
(Seconds: ,790)
(NumCarts: ,9)
(Seconds: ,791)
(NumCarts: ,9)
(Seconds: ,792)
(NumCarts: ,9)
(Seconds: ,793)
(NumCarts: ,9)
(Seconds: ,794)
(NumCarts: ,9)
(Seconds: ,795)
(NumCarts: ,9)
(Seconds: ,796)
(NumCarts: ,9)
(Seconds: ,797)
(NumCarts: ,9)
(Seconds: ,798)
(NumCarts: ,9)
(Seconds: ,799)
(NumCarts: ,9)
(Seconds: ,800)
(NumCarts: ,9)
(Seconds: ,801)
(NumCarts: ,9)
(Seconds: ,802)
(NumCarts: ,7)
(Seconds: ,803)
(NumCarts: ,7)
(Seconds: ,804)
(NumCarts: ,7)
(Seconds: ,805)
(NumCarts: ,7)
(Seconds: ,806)
(NumCarts: ,7)
(Seconds: ,807)
(NumCarts: ,7)
(Seconds: ,808)
(NumCarts: ,7)
(Seconds: ,809)
(NumCarts: ,7)
(Seconds: ,810)
(NumCarts: ,7)
(Seconds: ,811)
(NumCarts: ,7)
(Seconds: ,812)
(NumCarts: ,7)
(Seconds: ,813)
(NumCarts: ,7)
(Seconds: ,814)
(NumCarts: ,7)
(Seconds: ,815)
(NumCarts: ,7)
(Seconds: ,816)
(NumCarts: ,7)
(Seconds: ,817)
(NumCarts: ,7)
(Seconds: ,818)
(NumCarts: ,7)
(Seconds: ,819)
(NumCarts: ,7)
(Seconds: ,820)
(NumCart

(NumCarts: ,5)
(Seconds: ,1055)
(NumCarts: ,5)
(Seconds: ,1056)
(NumCarts: ,5)
(Seconds: ,1057)
(NumCarts: ,5)
(Seconds: ,1058)
(NumCarts: ,5)
(Seconds: ,1059)
(NumCarts: ,5)
(Seconds: ,1060)
(NumCarts: ,5)
(Seconds: ,1061)
(NumCarts: ,5)
(Seconds: ,1062)
(NumCarts: ,5)
(Seconds: ,1063)
(NumCarts: ,5)
(Seconds: ,1064)
(NumCarts: ,5)
(Seconds: ,1065)
(NumCarts: ,5)
(Seconds: ,1066)
(NumCarts: ,5)
(Seconds: ,1067)
(NumCarts: ,5)
(Seconds: ,1068)
(NumCarts: ,5)
(Seconds: ,1069)
(NumCarts: ,5)
(Seconds: ,1070)
(NumCarts: ,5)
(Seconds: ,1071)
(NumCarts: ,5)
(Seconds: ,1072)
(NumCarts: ,5)
(Seconds: ,1073)
(NumCarts: ,5)
(Seconds: ,1074)
(NumCarts: ,5)
(Seconds: ,1075)
(NumCarts: ,5)
(Seconds: ,1076)
(NumCarts: ,5)
(Seconds: ,1077)
(NumCarts: ,5)
(Seconds: ,1078)
(NumCarts: ,5)
(Seconds: ,1079)
(NumCarts: ,5)
(Seconds: ,1080)
(NumCarts: ,5)
(Seconds: ,1081)
(NumCarts: ,5)
(Seconds: ,1082)
(NumCarts: ,5)
(Seconds: ,1083)
(NumCarts: ,5)
(Seconds: ,1084)
(NumCarts: ,5)
(Seconds: ,1085)
(NumCart

...

(Seconds: ,9916)
(NumCarts: ,3)
(Seconds: ,9917)
(NumCarts: ,3)
(Seconds: ,9918)
(NumCarts: ,3)
(Seconds: ,9919)
(NumCarts: ,3)
(Seconds: ,9920)
(NumCarts: ,3)
(Seconds: ,9921)
(NumCarts: ,3)
(Seconds: ,9922)
(NumCarts: ,3)
(Seconds: ,9923)
(NumCarts: ,3)
(Seconds: ,9924)
(NumCarts: ,3)
(Seconds: ,9925)
(NumCarts: ,3)
(Seconds: ,9926)
(NumCarts: ,3)
(Seconds: ,9927)
(NumCarts: ,3)
(Seconds: ,9928)
(NumCarts: ,3)
(Seconds: ,9929)
(NumCarts: ,3)
(Seconds: ,9930)
(NumCarts: ,3)
(Seconds: ,9931)
(NumCarts: ,3)
(Seconds: ,9932)
(NumCarts: ,3)
(Seconds: ,9933)
(NumCarts: ,3)
(Seconds: ,9934)
(NumCarts: ,3)
(Seconds: ,9935)
(NumCarts: ,3)
(Seconds: ,9936)
(NumCarts: ,3)
(Seconds: ,9937)
(NumCarts: ,3)
(Seconds: ,9938)
(NumCarts: ,3)
(Seconds: ,9939)
(NumCarts: ,3)
(Seconds: ,9940)
(NumCarts: ,3)
(Seconds: ,9941)
(NumCarts: ,3)
(Seconds: ,9942)
(NumCarts: ,3)
(Seconds: ,9943)
(NumCarts: ,3)
(Seconds: ,9944)
(NumCarts: ,3)
(Seconds: ,9945)
(NumCarts: ,3)
(Seconds: ,9946)
(NumCarts: ,3)
(Seconds

(Seconds: ,10174)
(NumCarts: ,3)
(Seconds: ,10175)
(NumCarts: ,3)
(Seconds: ,10176)
(NumCarts: ,3)
(Seconds: ,10177)
(NumCarts: ,3)
(Seconds: ,10178)
(NumCarts: ,3)
(Seconds: ,10179)
(NumCarts: ,3)
(Seconds: ,10180)
(NumCarts: ,3)
(Seconds: ,10181)
(NumCarts: ,3)
(Seconds: ,10182)
(NumCarts: ,3)
(Seconds: ,10183)
(NumCarts: ,3)
(Seconds: ,10184)
(NumCarts: ,3)
(Seconds: ,10185)
(NumCarts: ,3)
(Seconds: ,10186)
(NumCarts: ,3)
(Seconds: ,10187)
(NumCarts: ,3)
(Seconds: ,10188)
(NumCarts: ,3)
(Seconds: ,10189)
(NumCarts: ,3)
(Seconds: ,10190)
(NumCarts: ,3)
(Seconds: ,10191)
(NumCarts: ,3)
(Seconds: ,10192)
(NumCarts: ,3)
(Seconds: ,10193)
(NumCarts: ,3)
(Seconds: ,10194)
(NumCarts: ,3)
(Seconds: ,10195)
(NumCarts: ,3)
(Seconds: ,10196)
(NumCarts: ,3)
(Seconds: ,10197)
(NumCarts: ,3)
(Seconds: ,10198)
(NumCarts: ,3)
(Seconds: ,10199)
(NumCarts: ,3)
(Seconds: ,10200)
(NumCarts: ,3)
(Seconds: ,10201)
(NumCarts: ,3)
(Seconds: ,10202)
(NumCarts: ,3)
(Seconds: ,10203)
(NumCarts: ,3)
(Seconds: 

(NumCarts: ,3)
(Seconds: ,10427)
(NumCarts: ,3)
(Seconds: ,10428)
(NumCarts: ,3)
(Seconds: ,10429)
(NumCarts: ,3)
(Seconds: ,10430)
(NumCarts: ,3)
(Seconds: ,10431)
(NumCarts: ,3)
(Seconds: ,10432)
(NumCarts: ,3)
(Seconds: ,10433)
(NumCarts: ,3)
(Seconds: ,10434)
(NumCarts: ,3)
(Seconds: ,10435)
(NumCarts: ,3)
(Seconds: ,10436)
(NumCarts: ,3)
(Seconds: ,10437)
(NumCarts: ,3)
(Seconds: ,10438)
(NumCarts: ,3)
(Seconds: ,10439)
(NumCarts: ,3)
(Seconds: ,10440)
(NumCarts: ,3)
(Seconds: ,10441)
(NumCarts: ,3)
(Seconds: ,10442)
(NumCarts: ,3)
(Seconds: ,10443)
(NumCarts: ,3)
(Seconds: ,10444)
(NumCarts: ,3)
(Seconds: ,10445)
(NumCarts: ,3)
(Seconds: ,10446)
(NumCarts: ,3)
(Seconds: ,10447)
(NumCarts: ,3)
(Seconds: ,10448)
(NumCarts: ,3)
(Seconds: ,10449)
(NumCarts: ,3)
(Seconds: ,10450)
(NumCarts: ,3)
(Seconds: ,10451)
(NumCarts: ,3)
(Seconds: ,10452)
(NumCarts: ,3)
(Seconds: ,10453)
(NumCarts: ,3)
(Seconds: ,10454)
(NumCarts: ,3)
(Seconds: ,10455)
(NumCarts: ,3)
(Seconds: ,10456)
(NumCarts:

(Seconds: ,10679)
(NumCarts: ,3)
(Seconds: ,10680)
(NumCarts: ,3)
(Seconds: ,10681)
(NumCarts: ,3)
(Seconds: ,10682)
(NumCarts: ,3)
(Seconds: ,10683)
(NumCarts: ,3)
(Seconds: ,10684)
(NumCarts: ,3)
(Seconds: ,10685)
(NumCarts: ,3)
(Seconds: ,10686)
(NumCarts: ,3)
(Seconds: ,10687)
(NumCarts: ,3)
(Seconds: ,10688)
(NumCarts: ,3)
(Seconds: ,10689)
(NumCarts: ,3)
(Seconds: ,10690)
(NumCarts: ,3)
(Seconds: ,10691)
(NumCarts: ,3)
(Seconds: ,10692)
(NumCarts: ,3)
(Seconds: ,10693)
(NumCarts: ,3)
(Seconds: ,10694)
(NumCarts: ,3)
(Seconds: ,10695)
(NumCarts: ,3)
(Seconds: ,10696)
(NumCarts: ,3)
(Seconds: ,10697)
(NumCarts: ,3)
(Seconds: ,10698)
(NumCarts: ,3)
(Seconds: ,10699)
(NumCarts: ,3)
(Seconds: ,10700)
(NumCarts: ,3)
(Seconds: ,10701)
(NumCarts: ,3)
(Seconds: ,10702)
(NumCarts: ,3)
(Seconds: ,10703)
(NumCarts: ,3)
(Seconds: ,10704)
(NumCarts: ,3)
(Seconds: ,10705)
(NumCarts: ,3)
(Seconds: ,10706)
(NumCarts: ,3)
(Seconds: ,10707)
(NumCarts: ,3)
(Seconds: ,10708)
(NumCarts: ,3)
(Seconds: 

res5_2: ListBuffer[(Int, Int)] = ListBuffer(
  (64, 57),
  (111, 100),
  (31, 57),
  (75, 87),
  (119, 28),
  (26, 74),
  (90, 56),
  (58, 42)
)

In [7]:
crashes
seconds
//printGrid(grid, carts, crashes)
carts

res6_0: ListBuffer[(Int, Int)] = ListBuffer(
  (64, 57),
  (111, 100),
  (31, 57),
  (75, 87),
  (119, 28),
  (26, 74),
  (90, 56),
  (58, 42)
)
res6_1: Int = 10830
res6_2: ListBuffer[Cart] = ListBuffer(v - SOUTH at (136, 8))